In [1]:
#Data import
import pandas as pd

evermindPath = "s3a://alpha-health-pro/amber/evermind/events/2020/*/*/*"
evermindDF = spark.read.json(evermindPath)
evermind_df = evermindDF.toPandas()

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [6]:
df = evermind_df.copy()
df['createdAt'] = pd.to_datetime(df.createdAt, utc = True)
df['prev_mytimestamp'] = df.groupby(['userId'])['createdAt'].transform(lambda x:x.shift(1))

In [7]:
df.head()

,createdAt,data,deploymentEnv,device,element,error,event,level,message,product,schema,source,stacktrace,type,unique_name,user,userId,view,prev_mytimestamp
0,2020-03-20 07:08:31.112956+00:00,None,prod,"(, iPhone12,5, iOS, d9bae436b55136042f7a310738...",verify_code,None,tap,None,None,"(com.wingmanalpha.app.evermind, Evermind, 2.4.1)",/schemas/events/analytics-device-event/0.json,analytics,None,track,tap.verify.verify_code,"(,)",,verify,NaT
1,2020-03-20 07:08:32.173944+00:00,"(None, None, None, None, None, None, None, Non...",prod,(ddb54e05c5a53b316166498ff0a832544fe16e0fad048...,None,None,screen_view,None,None,"(com.wingmanalpha.app.evermind, Evermind, 2.4.1)",/schemas/events/analytics-device-event/0.json,analytics,None,track,screen_view.welcome.null,(28a80fdb1793dc30b2c5d81555cf8e4f765f6aa5b9c01...,28a80fdb1793dc30b2c5d81555cf8e4f765f6aa5b9c01b...,welcome,NaT
2,2020-03-20 07:08:36.454686+00:00,"(None, None, None, None, None, None, None, Non...",prod,(ddb54e05c5a53b316166498ff0a832544fe16e0fad048...,start_assessment,None,tap,None,None,"(com.wingmanalpha.app.evermind, Evermind, 2.4.1)",/schemas/events/analytics-device-event/0.json,analytics,None,track,tap.welcome.start_assessment,(28a80fdb1793dc30b2c5d81555cf8e4f765f6aa5b9c01...,28a80fdb1793dc30b2c5d81555cf8e4f765f6aa5b9c01b...,welcome,2020-03-20 07:08:32.173944+00:00
3,2020-03-20 07:08:36.455877+00:00,"(None, None, None, None, None, None, None, Non...",prod,(ddb54e05c5a53b316166498ff0a832544fe16e0fad048...,None,None,screen_view,None,None,"(com.wingmanalpha.app.evermind, Evermind, 2.4.1)",/schemas/events/analytics-device-event/0.json,analytics,None,track,screen_view.assessment.null,(28a80fdb1793dc30b2c5d81555cf8e4f765f6aa5b9c01...,28a80fdb1793dc30b2c5d81555cf8e4f765f6aa5b9c01b...,assessment,2020-03-20 07:08:36.454686+00:00
4,2020-03-20 07:08:33.916720+00:00,None,prod,"(, iPhone11,8, iOS, e2a479f13d015870287e0eabd5...",create_account,None,tap,None,None,"(com.wingmanalpha.app.evermind, Evermind, 2.4.1)",/schemas/events/analytics-device-event/0.json,analytics,None,track,tap.sign_up.create_account,"(,)",,sign_up,2020-03-20 07:08:31.112956+00:00


In [9]:
from datetime import timedelta

T = timedelta(seconds=30*60)

df['new_session'] = ((df['createdAt']
                            - pd.to_datetime(df.prev_mytimestamp))>=T).astype(int)


In [10]:
df['increment'] = df.groupby("userId")['new_session'].cumsum()
df['session_id'] = df['userId'].astype(str) + '_' + df['increment'].astype(str)
df = df.sort_values(['createdAt'])

In [14]:
df[df['userId'] == '41e1dcca-20c7-4027-b36b-4386740a535b']['session_id'].sort_values().unique()

array(['41e1dcca-20c7-4027-b36b-4386740a535b_0',
       '41e1dcca-20c7-4027-b36b-4386740a535b_1'], dtype=object)

## Which activities with which PSS?

In [32]:
import re
def getKey(k, raw):
    """
    Function used to return the content for different keys in the data array. 
    This function will be used to create new columns for scores etc.
    """
    if isinstance(raw, float):
        return(float('NaN'))
    
    raw = str(raw).replace('null', 'NaN')
    
    if k == 'score':
        num = re.search('{}=(\d+)'.format(k), raw)
        if num:
            num = int(num.group(1))
            return(num)

    if k == 'answers': 
        ary = re.search(r"\[(.*)\]".format(k), raw)
        print(ary)
        if ary:
            ary = ary.group()
            ary = re.sub('\D', ' ', ary)
            ary = ary.split()
            return(ary)
    
    # if string
    sub_string = re.search('{}=(.+?),'.format(k), raw)
    if sub_string:
        string = sub_string.group(1)
        string = string.replace("'", "")
        return(string)


In [20]:
df2 = df.copy()
df2.fillna(value = pd.np.nan, inplace=True)
general = df2['data']
df2['score'] = general.apply(lambda x: getKey('score', x))

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [21]:
df2

,createdAt,data,deploymentEnv,device,element,error,event,level,message,product,...,type,unique_name,user,userId,view,prev_mytimestamp,new_session,increment,session_id,score
106844,2019-11-23 10:09:56.012798+00:00,NaN,stage,(bbf1f4323c65e7056ae3c3dbfe5729a5a276684baf157...,NaN,NaN,app_pause,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 1.0....",...,track,app_pause.home.null,(b612c4ae58ecf05cbabdc563170f76ee4b955f9f246ce...,b612c4ae58ecf05cbabdc563170f76ee4b955f9f246cef...,home,2020-03-22 17:21:59.856723+00:00,0,1,b612c4ae58ecf05cbabdc563170f76ee4b955f9f246cef...,NaN
78795,2019-11-23 10:09:56.012798+00:00,NaN,stage,(bbf1f4323c65e7056ae3c3dbfe5729a5a276684baf157...,NaN,NaN,app_pause,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 1.0....",...,track,app_pause.home.null,(b612c4ae58ecf05cbabdc563170f76ee4b955f9f246ce...,b612c4ae58ecf05cbabdc563170f76ee4b955f9f246cef...,home,NaT,0,0,b612c4ae58ecf05cbabdc563170f76ee4b955f9f246cef...,NaN
108918,2019-11-27 09:09:37.038397+00:00,NaN,prod,(2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef3...,ok,NaN,tap,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 1.0.1)",...,track,tap.sign-up.ok,(2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef3...,2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef37...,sign-up,NaT,0,0,2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef37...,NaN
108919,2019-11-27 09:09:39.480988+00:00,NaN,prod,(2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef3...,NaN,NaN,app_pause,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 1.0.1)",...,track,app_pause.sign-up.null,(2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef3...,2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef37...,sign-up,2019-11-27 09:09:37.038397+00:00,0,0,2f8f276daeda4df0811a7a9a720c3fd24450ff8723ef37...,NaN
106488,2019-11-29 20:18:02.235500+00:00,"(None, None, None, None, None, None, None, Non...",prod,"(89406779-94FE-4E47-B6A0-08C60C16584B, iPhone1...",back,NaN,tap,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 1.0.1)",...,track,tap.module-overview.back,"(4c0831d9-6ad4-4cef-be58-3a510a6b602c,)",4c0831d9-6ad4-4cef-be58-3a510a6b602c,module-overview,NaT,0,0,4c0831d9-6ad4-4cef-be58-3a510a6b602c_0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108864,2020-04-07 07:33:23.774207+00:00,NaN,prod,"(, iPhone8,4, iOS, 085524224ee9a2561f63f5ad743...",ok,NaN,tap,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 2.4.0)",...,track,tap.sign_up.ok,"(,)",,sign_up,2020-04-07 07:33:17.707323+00:00,0,850,_850,NaN
108865,2020-04-07 07:33:30.406767+00:00,NaN,prod,"(, iPhone8,4, iOS, 085524224ee9a2561f63f5ad743...",create_account,NaN,tap,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 2.4.0)",...,track,tap.sign_up.create_account,"(,)",,sign_up,2020-04-07 07:33:23.774207+00:00,0,850,_850,NaN
108866,2020-04-07 07:33:38.506954+00:00,NaN,prod,"(, iPhone8,4, iOS, 085524224ee9a2561f63f5ad743...",ok,NaN,tap,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 2.4.0)",...,track,tap.sign_up.ok,"(,)",,sign_up,2020-04-07 07:33:30.406767+00:00,0,850,_850,NaN
108867,2020-04-07 07:33:40.242394+00:00,NaN,prod,"(, iPhone8,4, iOS, 085524224ee9a2561f63f5ad743...",create_account,NaN,tap,NaN,NaN,"(com.wingmanalpha.app.evermind, Evermind, 2.4.0)",...,track,tap.sign_up.create_account,"(,)",,sign_up,2020-04-07 07:33:38.506954+00:00,0,850,_850,NaN


In [23]:
max = []
max = df2.groupby('session_id')['createdAt'].max()

min = []
min = df2.groupby('session_id')['createdAt'].min()

diff = max - min
diff

session_id
003485c5fc50ee1686c0b837634d2e1263cd483147a7f780691bcd72991c6fa2_0    0 days 00:00:03.413478
003485c5fc50ee1686c0b837634d2e1263cd483147a7f780691bcd72991c6fa2_1   28 days 21:46:20.138066
003485c5fc50ee1686c0b837634d2e1263cd483147a7f780691bcd72991c6fa2_2   14 days 17:37:10.352896
003485c5fc50ee1686c0b837634d2e1263cd483147a7f780691bcd72991c6fa2_3    0 days 00:00:04.646377
003485c5fc50ee1686c0b837634d2e1263cd483147a7f780691bcd72991c6fa2_4    0 days 00:00:40.819964
                                                                               ...          
fe5897458a09cbda336abb88018ba6e4570d570fc935c5e0b036453068e74363_3           0 days 00:00:00
fe5897458a09cbda336abb88018ba6e4570d570fc935c5e0b036453068e74363_4    0 days 00:00:38.033799
fe5897458a09cbda336abb88018ba6e4570d570fc935c5e0b036453068e74363_5   13 days 23:33:37.289543
fe97d39f891deba49b10722ab251c16ab7e3220f4eaf5e47f5d4189a967d2926_0    0 days 00:01:36.797858
ffc6bcea4ef237480bca286bc527ce56f413ac4488ca47d6d9d1f75a7ba

In [24]:
df2['activity-title'] = general.apply(lambda x: getKey('activity-title', x))

In [25]:
df2 = df2.replace('null', pd.np.nan)

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


In [26]:
def outputPerUser(df, cat):
    df.dropna(subset = [cat], inplace=True)
    
    df.sort_values(["session_id", cat])
    valid_ids = df['session_id'].unique()
    new = df[(df['session_id'].isin(valid_ids))]
    new = new.groupby('session_id')
    new = new[cat, 'activity-title'].agg(list) 
    return new

In [27]:
scoresdf = outputPerUser(df2, 'score')

/Users/yc00123/.virtualenvs/py3.6.8-data-access/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [28]:
scoresdf.sort_values('score')

,score,activity-title
session_id,,
2397e1e3-e7b4-459a-82c6-c8a69cdc7031_2,[0.0],['Match the shapes']
1513c7187ae4eb70a3072c73dfb19d966b263bec29d408a7bbbe12901a0aacd1_0,[0.0],[None]
46dc9b12-42c4-4ba7-8c1c-9d1404e0903a_2,[0.0],['Match the shapes']
2397e1e3-e7b4-459a-82c6-c8a69cdc7031_1,"[0.0, 0.0, 0.0]","['Match the shapes', 'Match the shapes', 'Matc..."
2397e1e3-e7b4-459a-82c6-c8a69cdc7031_5,"[0.0, 0.0, 0.0, 0.0]","['Match the shapes', 'Match the shapes', 'Matc..."
...,...,...
2f94c39761f39f59d1e9dac450cb62406751a22db1f46662af9150149287c15c_20,[32.0],[None]
fca2080eb6d516b2b582554528d6624464870eeff282f0cd73744e73058aa9e3_1,"[33.0, 33.0]","[None, None]"
a27b00f550d1151e2702c9bd177edb2ec78dea7edefe3e036d544febf7b8d09b_0,[34.0],[None]


In [29]:
import statistics 

def meanScore(x):
    m = statistics.mean(x)
    return m

In [30]:
general = scoresdf['score']
scoresdf['meanScore'] = general.apply(lambda x: meanScore(x))

In [31]:
scoresdf

,score,activity-title,meanScore
session_id,,,
003485c5fc50ee1686c0b837634d2e1263cd483147a7f780691bcd72991c6fa2_2,"[13.0, 16.0]","[None, None]",14.5
008c8cd109af590b33bc599e81e5212f699c870ba448de18333b57fd9c2a4e50_0,"[20.0, 20.0]","[None, None]",20.0
014f2740692fd10525909388506f48ca43dcf34066c29ae08d10aff1b62036f8_0,[20.0],[None],20.0
02064ef9ea1ee046cf82d94530b11ac134299a641577e1676d7b633813e44e78_0,[20.0],[None],20.0
02cce9482d12301aad3409374eb7c91da3347c41a61ad552e48bdadf11fa40f7_0,[15.0],[None],15.0
...,...,...,...
fd9890bc631b19514d3e93b4a9e17e39517665ba4808a1723cd4f2fab4205217_1,[14.0],[None],14.0
fdd17fc63dd94bac507250c09af62acdb270b7a96383294b227902baa2c8194d_0,[12.0],[None],12.0
fe5897458a09cbda336abb88018ba6e4570d570fc935c5e0b036453068e74363_2,[21.0],[None],21.0
